<a href="https://colab.research.google.com/github/Veronica-Campana/prog/blob/main/data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Python code to perform data augmentation, exploiting Albumentation library.
# Three transformations will be applied: horizontal flip, vertical flip, random brightness change.

# Install Albumentations library
! pip install --upgrade albumentations
import albumentations as A
import cv2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 116 kB 5.1 MB/s 
     |████████████████████████████████| 48.3 MB 1.9 MB/s 
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
# Mount the drive into Colab in order to upload the dataset to be augmented
from google.colab import drive
drive.mount('/content/gdrive')

# Create a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive 

Mounted at /content/gdrive
 bright			   objs
'Colab Notebooks'	   obj.zip
 data_augmentation.ipynb   OneDrive_1_24-7-2022.zip
 DATASET		  'Papiro Giulia.gdoc'
 DATASET_AUGM_DIVISO	  'Papiro Lucia.gdoc'
 DATASET_AUGM_DIVISO.zip   pred_mask
 DATASET_AUGMENTATION	   segmented_hands
 DATASET_HANDSONFACE	   TESTDATASET
 DATASET_HANDSONFACE.zip   TESTDATASET.zip
 DATASET.zip		   testobjs
 DAT_HANDOVERFACE_DEF	   test.zip
 EgoHands		   vertic_flip
 HANDOVERFACE		   yolov4
 models			   yolov4-custom_3000.weights
'My Drive'		   yolov4-custom_4000.weights
 Notability		   yolov4-custom.cfg
 obj.data		   yolov4-custom_last.weights
 obj.names


In [ ]:
# Unzip DATASET_AUGMENTATION, that contains 320 images with corresponding bounding boxes.
# The annotations have been obtained using LabelImg, in YOLO format.
%cd /mydrive
!unzip /mydrive/DATASET_AUGMENTATION.zip

/content/gdrive/My Drive
Archive:  /mydrive/DATASET_AUGMENTATION.zip
   creating: DATASET_AUGMENTATION/BB/
  inflating: DATASET_AUGMENTATION/BB/10.txt  
  inflating: DATASET_AUGMENTATION/BB/100.txt  
  inflating: DATASET_AUGMENTATION/BB/101.txt  
  inflating: DATASET_AUGMENTATION/BB/102.txt  
  inflating: DATASET_AUGMENTATION/BB/103.txt  
  inflating: DATASET_AUGMENTATION/BB/104.txt  
  inflating: DATASET_AUGMENTATION/BB/105.txt  
  inflating: DATASET_AUGMENTATION/BB/106.txt  
  inflating: DATASET_AUGMENTATION/BB/107.txt  
  inflating: DATASET_AUGMENTATION/BB/108.txt  
  inflating: DATASET_AUGMENTATION/BB/109.txt  
  inflating: DATASET_AUGMENTATION/BB/11.txt  
  inflating: DATASET_AUGMENTATION/BB/110.txt  
  inflating: DATASET_AUGMENTATION/BB/111.txt  
  inflating: DATASET_AUGMENTATION/BB/112.txt  
  inflating: DATASET_AUGMENTATION/BB/114.txt  
  inflating: DATASET_AUGMENTATION/BB/115.txt  
  inflating: DATASET_AUGMENTATION/BB/116.txt  
  inflating: DATASET_AUGMENTATION/BB/117.txt  
  

In [ ]:
from natsort import natsorted
import os
import numpy as np
import random

In [ ]:
def dir_list(d):
    return [os.path.join(d,f) for f in os.listdir(d)] 

In [ ]:
# Save paths to images and bounding boxes
testbb_path = natsorted(dir_list('/mydrive/DATASET_AUGMENTATION/BB'))
testimage_path = natsorted(dir_list('/mydrive/DATASET_AUGMENTATION/IM'))

# Check if the number of elements in testbb_path and testimage_path is the same. If not, print an error
if (len(testbb_path) != len(testimage_path)):
  print('Error: not same number of images and bounding boxes')

# List all bounding boxes for each image in testimage_path. Create a list of lists.
# Each list contains the bounding boxes, relative to one of the images
testbb = []*len(testbb_path)
for testpath in testbb_path:
  testbb_img = []
  file = open(testpath, 'r')
  lines = file.readlines()
  for line in lines:
    testbb_img.append([float(x) for x in line.split()])   
  testbb.append(testbb_img) 

In [ ]:
# Obtain bounding boxes without class (eliminate initial 0 in YOLO format)
testbb_noclass = []*len(testbb_path)
for i in range(len(testbb_path)):
    image = cv2.imread(testimage_path[i])

    bb_image = []
    for boxes in testbb[i]:
        bb_image.append(boxes[1:5]) # Take only the coordinates of the bounding boxes

    testbb_noclass.append(bb_image)
print(testbb_noclass) # Check if 0 has been removed correctly

[[[0.559942, 0.533626, 0.178363, 0.24269]], [[0.1875, 0.279444, 0.371667, 0.496667], [0.664167, 0.568333, 0.485, 0.403333]], [[0.374601, 0.543165, 0.091054, 0.347722]], [[0.14, 0.724044, 0.258182, 0.551913]], [[0.598856, 0.175245, 0.138889, 0.242647]], [[0.46849, 0.562083, 0.143229, 0.334167], [0.523177, 0.58375, 0.096354, 0.2975]], [[0.442727, 0.738356, 0.147273, 0.375342]], [[0.285185, 0.496667, 0.362963, 0.58], [0.67037, 0.525, 0.402469, 0.56]], [[0.291815, 0.513889, 0.320285, 0.683333]], [[0.217391, 0.53869, 0.160535, 0.363095]], [[0.304878, 0.458667, 0.260163, 0.362667], [0.477642, 0.455333, 0.365854, 0.414667]], [[0.252778, 0.56875, 0.294444, 0.4375]], [[0.251309, 0.410985, 0.492147, 0.708333], [0.811518, 0.393939, 0.376963, 0.636364]], [[0.327215, 0.458, 0.358228, 0.588]], [[0.352439, 0.469512, 0.397561, 0.54878]], [[0.514706, 0.589461, 0.156863, 0.443627]], [[0.59625, 0.424015, 0.1775, 0.405253], [0.83375, 0.442777, 0.135, 0.409006]], [[0.30719, 0.24, 0.27451, 0.373333]], [[0.6

In [ ]:
# HORIZONTAL FLIP

# Create the directory horiz_flip, where images and bounding boxes will be saved after applying horizontal flip
%cd /mydrive
!mkdir horiz_flip
%cd /mydrive/horiz_flip

/content/gdrive/My Drive
/content/gdrive/My Drive/horiz_flip


In [ ]:
for j in range(len(testbb_path)):
  # Read each image with corresponding list of bounding boxes
  image = cv2.imread(testimage_path[j])
  bboxes = testbb_noclass[j]

  # Apply horizontal flip
  transform = A.Compose([
    A.HorizontalFlip(p=0.5)], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))
  random.seed(7)
  class_labels = []
  for i in range(len(bboxes)):
    class_labels.append('hand')

  transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
  transformed_image = transformed['image']
  transformed_bboxes = transformed['bboxes']
  transformed_class_labels = transformed['class_labels']

  # Save flipped images and text files into directory "horiz_flip"
  filenameImg = "hflip_"+str(j)+".jpg";
  cv2.imwrite(filenameImg, transformed_image)

  filenameBb = "hflip_"+str(j)+".txt";
  f = open(filenameBb,"w+")

  bb_image = []
  for boxes in transformed_bboxes:
      # Save transformed bounding boxes in .txt file, where each line is associated with a different bounding box and the first element indicates the class
      f.write("0 ")
      for items in boxes: 
        f.write(str(items) + " ")
      f.write("\n")

  f.close()

In [ ]:
# VERTICAL FLIP

# Create the directory vertic_flip, where images and bounding boxes will be saved after applying vertical flip
%cd /mydrive
!mkdir vertic_flip
%cd /mydrive/vertic_flip

/content/gdrive/My Drive
/content/gdrive/My Drive/vertic_flip


In [ ]:
for j in range(len(testbb_path)):
  # Read each image with corresponding list of bounding boxes
  image = cv2.imread(testimage_path[j])
  bboxes = testbb_noclass[j]

  # Apply vertical flip
  transform = A.Compose([
    A.VerticalFlip(p=0.5)], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))
  random.seed(7)
  class_labels = []
  for i in range(len(bboxes)):
    class_labels.append('hand')

  transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
  transformed_image = transformed['image']
  transformed_bboxes = transformed['bboxes']
  transformed_class_labels = transformed['class_labels']

  # Save flipped images and text files into directory "vertic_flip"
  filenameImg = "vflip_"+str(j)+".jpg";
  cv2.imwrite(filenameImg, transformed_image)

  filenameBb = "vflip_"+str(j)+".txt";
  f = open(filenameBb,"w+")

  bb_image = []
  for boxes in transformed_bboxes:
      # Save transformed bounding boxes in .txt file, where each line is associated with a different bounding box and the first element indicates the class
      f.write("0 ")
      for items in boxes: 
        f.write(str(items) + " ")
      f.write("\n")

  f.close()

In [ ]:
# CHANGE BRIGHTNESS

# Create the directory bright, where images and bounding boxes will be saved after applying random change of brightness
%cd /mydrive
!mkdir bright
%cd /mydrive/bright

/content/gdrive/My Drive
/content/gdrive/My Drive/bright


In [ ]:
for j in range(len(testbb_path)):
  # Read each image with corresponding list of bounding boxes
  image = cv2.imread(testimage_path[j])
  bboxes = testbb_noclass[j]

  # Apply random change of brightness
  transform = A.Compose([
    A.RandomBrightnessContrast(p=0.3)], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))
  random.seed(7)
  class_labels = []
  for i in range(len(bboxes)):
    class_labels.append('hand')

  transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
  transformed_image = transformed['image']
  transformed_bboxes = transformed['bboxes']
  transformed_class_labels = transformed['class_labels']

  # Save transformed images and text files into directory "bright"
  filenameImg = "bright_"+str(j)+".jpg";
  cv2.imwrite(filenameImg, transformed_image)

  filenameBb = "bright_"+str(j)+".txt";
  f = open(filenameBb,"w+")

  bb_image = []
  for boxes in transformed_bboxes:
      # Save transformed bounding boxes in .txt file, where each line is associated with a different bounding box and the first element indicates the class
      f.write("0 ")
      for items in boxes: 
        f.write(str(items) + " ")
      f.write("\n")

  f.close()